In [9]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [10]:
mem_data = pd.read_csv('data/mem_data.csv')
mem_tr = pd.read_csv('data/mem_transaction.csv')

mem_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10764 entries, 0 to 10763
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MEM_ID       10764 non-null  int64  
 1   M_STORE_ID   10764 non-null  int64  
 2   GENDER       10764 non-null  object 
 3   BIRTH_DT     5997 non-null   object 
 4   BIRTH_SL     10764 non-null  object 
 5   ZIP_CD       10764 non-null  object 
 6   RGST_DT      10764 non-null  object 
 7   VISIT_CNT    10764 non-null  float64
 8   SALES_AMT    10764 non-null  float64
 9   LAST_VST_DT  10764 non-null  object 
 10  USABLE_PNT   10764 non-null  float64
 11  USED_PNT     10764 non-null  float64
 12  ACC_PNT      10764 non-null  float64
 13  USABLE_INIT  10764 non-null  float64
 14  SMS          10764 non-null  object 
dtypes: float64(6), int64(2), object(7)
memory usage: 1.2+ MB


## **[mem_data 처리]**

**[변수 처리 1]** SMS 수신동의 정수 처리하기

In [11]:
mem_data.SMS.value_counts()

Y    10575
N      189
Name: SMS, dtype: int64

In [12]:
mem_data.SMS = (mem_data.SMS=='Y').astype(int)
mem_data.SMS.value_counts()

1    10575
0      189
Name: SMS, dtype: int64

**[변수 처리 2]** 양/음력(BIRTH_SL) 정수 처리하기 

In [13]:
mem_data.BIRTH_SL.value_counts()

S    8973
L    1791
Name: BIRTH_SL, dtype: int64

In [14]:
mem_data.BIRTH_SL = (mem_data.BIRTH_SL=='S').astype(int)
mem_data.BIRTH_SL.value_counts()

1    8973
0    1791
Name: BIRTH_SL, dtype: int64

**[변수 처리 3]** 구매 합계(SALES_AMT) 로그 처리 하기
- 참고 : 로그처리에는 음수가 들어갈 수 없음 / 0이 있을 경우 1을 더하고 처리

**[변수 처리 4]** 로그 처리가 필요하다고 생각되는 변수를 로그처리 하기

In [15]:
mem_data.SALES_AMT = np.log1p(mem_data.SALES_AMT)

In [16]:
cols = ['USABLE_PNT', 'USED_PNT', 'ACC_PNT', 'USABLE_INIT']

for col in cols:
    mem_data[col] = np.log1p(mem_data[col])

**[변수 생성 1]** 최근 방문 일자(LAST_VST_DT)로부터 경과일 구하기

In [17]:
f = pd.to_datetime(mem_data.LAST_VST_DT)
f = (pd.to_datetime('2007-12-31') - f).dt.days
mem_data['E_DAY'] = f
mem_data.E_DAY.head()

0    345
1     73
2    214
3     75
4    280
Name: E_DAY, dtype: int64

**[변수 생성 2]** 등록일(RGST_DT)로부터 경과일 구하기

In [18]:
f = pd.to_datetime(mem_data.RGST_DT)
f = (pd.to_datetime('2007-12-31') - f).dt.days
mem_data['R_DAY'] = f
mem_data.R_DAY.head()

0    694
1    694
2    694
3    694
4    694
Name: R_DAY, dtype: int64

**[변수 생성 3]** 우편번호(ZIP_CD)에서 광역행정구역 데이터 가져오기

In [19]:
f = [x[0] for x in mem_data.ZIP_CD]
mem_data['R_REGION'] = f
mem_data.R_REGION = mem_data.R_REGION.where(mem_data.R_REGION != '-', other=0).astype(int)
mem_data.R_REGION.head()

0    0
1    4
2    0
3    4
4    0
Name: R_REGION, dtype: int32

**[변수 생성 4]** 우편번호(ZIP_CD)에서 광역행정구역과 구를 합쳐서 데이터 가져오기

In [20]:
f = [x[0:2] for x in mem_data.ZIP_CD]
mem_data['R_DET'] = f
mem_data.R_DET = mem_data.R_DET.where(mem_data.R_DET != '-', other=0).astype(int)
mem_data.R_DET.head()

0     0
1    42
2     0
3    48
4     0
Name: R_DET, dtype: int32

## **[다른 데이터와 연동하여 변수 생성]**

**[변수 생성 1]** 평균 구매액 구하기

In [21]:
f = mem_tr.groupby('MEM_ID')['SELL_AMT'].agg('mean').reset_index()
f.columns = ['MEM_ID','SELL_AMT']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

**[변수 생성 2]** 포인트 적립 횟수 구하기

In [22]:
f = mem_tr[mem_tr.MEMP_TP=='A'].groupby('MEM_ID')['SELL_AMT'].agg('size').reset_index()
f.columns = ['MEM_ID','POINT_N']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

**[변수 생성 3]** 요일 구매 패턴 구하기 : 주중형 / 주말형

In [23]:
mem_tr.SELL_DT = pd.to_datetime(mem_tr.SELL_DT, format='%Y%m%d')
mem_tr['weekend'] = mem_tr.SELL_DT.dt.dayofweek >4
f = mem_tr.groupby(['MEM_ID','weekend'])['SELL_AMT'].agg('count')
f = f.unstack().fillna(0).astype('int').reset_index()
#reset_index 보다 가장 마지막 인덱스를 열 이름으로 사용하여 True, False로 구분 #결측치를 0으로 바꾸고 type을 횟수이므로 int로 변환
#마지막에 reset_index()
f.columns = ['MEM_ID','주중구매횟수','주말구매횟수']
f['patternofday'] = f.주중구매횟수 >= f.주말구매횟수
f.patternofday = f.patternofday.astype(int) #1이 주중형
f

,MEM_ID,주중구매횟수,주말구매횟수,patternofday
0,957,3,0,1
1,1054,5,0,1
2,1067,0,1,0
3,1068,8,1,1
4,1078,3,2,1
...,...,...,...,...
10759,2007042901468,0,1,0
10760,2007061900497,1,0,1
10761,2007062900420,2,0,1
10762,2007090300653,3,0,1


In [24]:
f_col = ['주중구매횟수','주말구매횟수']
f = f.drop(f_col, axis=1)
f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10764 entries, 0 to 10763
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   MEM_ID        10764 non-null  int64
 1   patternofday  10764 non-null  int32
dtypes: int32(1), int64(1)
memory usage: 126.3 KB


In [25]:
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

**[변수 생성 4]** 시간대별 표인트 적립 건수 구하기 : Morning(09-12) / Afternoon(13-17) / Evening(18-20)

In [26]:
mem_tr[mem_tr.MEMP_TP == 'A']
mem_tr['time'] = mem_tr.MEMP_DT.str[11:13]
mem_tr.head()

,STORE_ID,SELL_DT,MEMP_STY,MEM_ID,MEMP_DT,SELL_AMT,MEMP_TP,weekend,time
0,125,2006-02-27,O,1225434,2006-02-27 13:57:44.750000000,15500,A,False,13
1,125,2006-03-03,O,1181938,2006-03-03 15:43:56.987000000,22600,A,False,15
2,125,2006-03-22,O,6093,2006-03-22 18:08:45.563000000,11000,A,False,18
3,125,2006-04-03,O,1398202,2006-04-03 13:36:27.040000000,14400,A,False,13
4,125,2006-04-18,O,1225434,2006-04-18 16:04:39.583000000,6600,A,False,16


In [27]:
mem_tr['morning'] = (mem_tr.time == '09')|(mem_tr.time == '10')|(mem_tr.time == '11')|(mem_tr.time == '12')
mem_tr['Afternoon'] = (mem_tr.time == '13')|(mem_tr.time == '14')|(mem_tr.time == '15')|(mem_tr.time == '16')|(mem_tr.time == '17')
mem_tr['Evening'] = (mem_tr.time == '18')|(mem_tr.time == '19')|(mem_tr.time == '20')
mem_tr.head(100)

,STORE_ID,SELL_DT,MEMP_STY,MEM_ID,MEMP_DT,SELL_AMT,MEMP_TP,weekend,time,morning,Afternoon,Evening
0,125,2006-02-27,O,1225434,2006-02-27 13:57:44.750000000,15500,A,False,13,False,True,False
1,125,2006-03-03,O,1181938,2006-03-03 15:43:56.987000000,22600,A,False,15,False,True,False
2,125,2006-03-22,O,6093,2006-03-22 18:08:45.563000000,11000,A,False,18,False,False,True
3,125,2006-04-03,O,1398202,2006-04-03 13:36:27.040000000,14400,A,False,13,False,True,False
4,125,2006-04-18,O,1225434,2006-04-18 16:04:39.583000000,6600,A,False,16,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
95,128,2006-08-12,O,669115,2006-08-12 17:51:10.117000000,22400,A,True,17,False,True,False
96,128,2006-08-13,O,718750,2006-08-13 14:47:13.473000000,6300,A,True,14,False,True,False
97,128,2006-08-19,O,718750,2006-08-19 20:46:24.967000000,5500,A,True,20,False,False,True
98,128,2006-08-21,O,718750,2006-08-21 17:36:15.173000000,8500,A,False,17,False,True,False


In [28]:
mem_tr.morning = mem_tr.morning.astype(int)
mem_tr.Afternoon = mem_tr.Afternoon.astype(int)
mem_tr.Evening = mem_tr.Evening.astype(int)
mem_tr

,STORE_ID,SELL_DT,MEMP_STY,MEM_ID,MEMP_DT,SELL_AMT,MEMP_TP,weekend,time,morning,Afternoon,Evening
0,125,2006-02-27,O,1225434,2006-02-27 13:57:44.750000000,15500,A,False,13,0,1,0
1,125,2006-03-03,O,1181938,2006-03-03 15:43:56.987000000,22600,A,False,15,0,1,0
2,125,2006-03-22,O,6093,2006-03-22 18:08:45.563000000,11000,A,False,18,0,0,1
3,125,2006-04-03,O,1398202,2006-04-03 13:36:27.040000000,14400,A,False,13,0,1,0
4,125,2006-04-18,O,1225434,2006-04-18 16:04:39.583000000,6600,A,False,16,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
60044,30102,2007-10-17,O,843013,2007-10-17 21:59:32.313000000,29400,A,False,21,0,0,0
60045,30102,2007-11-04,O,843013,2007-11-04 18:01:50.217000000,11800,A,True,18,0,0,1
60046,99953,2006-02-24,O,1245048,2006-02-24 19:19:58.050000000,3300,A,False,19,0,0,1
60047,99953,2006-02-25,O,850156,2006-02-25 17:40:41.963000000,4400,A,True,17,0,1,0


In [29]:
f = mem_tr.groupby(['MEM_ID'])['morning'].agg('sum').reset_index()
f.columns = ['MEM_ID','Morning']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,Morning
0,957,0
1,1054,1
2,1067,0
3,1068,0
4,1078,0
...,...,...
10759,2007042901468,0
10760,2007061900497,0
10761,2007062900420,0
10762,2007090300653,0


In [30]:
f = mem_tr.groupby(['MEM_ID'])['Afternoon'].agg('sum').reset_index()
f.columns = ['MEM_ID','Afternoon']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,Afternoon
0,957,0
1,1054,2
2,1067,0
3,1068,4
4,1078,1
...,...,...
10759,2007042901468,0
10760,2007061900497,1
10761,2007062900420,0
10762,2007090300653,0


In [31]:
f = mem_tr.groupby(['MEM_ID'])['Evening'].agg('sum').reset_index()
f.columns = ['MEM_ID','Evening']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,Evening
0,957,2
1,1054,1
2,1067,0
3,1068,3
4,1078,3
...,...,...
10759,2007042901468,0
10760,2007061900497,0
10761,2007062900420,1
10762,2007090300653,3


**[변수 생성 5]** Groupby Aggregation Function 활용

In [32]:
f = mem_tr.groupby('MEM_ID')['SELL_AMT'].agg([('최대구매액', np.max)]).reset_index()
f.columns = ['MEM_ID','SELL_MAX']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [33]:
f = mem_tr.groupby('MEM_ID')['SELL_DT'].agg([('구매주기', lambda x: int((x.max() - x.min()).days / x.astype('str').str[:10].nunique()))]).reset_index()
f.columns = ['MEM_ID','Shop_Period']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,Shop_Period
0,957,110
1,1054,145
2,1067,0
3,1068,75
4,1078,108
...,...,...
10759,2007042901468,0
10760,2007061900497,0
10761,2007062900420,3
10762,2007090300653,8


In [34]:
visits = mem_tr.groupby('MEM_ID')['SELL_DT'].agg(lambda x: x.nunique())
f = (mem_tr.groupby('MEM_ID')['SELL_AMT'].sum() / visits).reset_index().rename(columns={0 : "내점당구매액"})
f.columns = ['MEM_ID','SELL_per']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,SELL_per
0,957,2500.000000
1,1054,47650.000000
2,1067,7700.000000
3,1068,12383.333333
4,1078,7375.000000
...,...,...
10759,2007042901468,3300.000000
10760,2007061900497,7700.000000
10761,2007062900420,5700.000000
10762,2007090300653,32033.333333


In [35]:
visits = mem_tr.groupby('MEM_ID')['SELL_DT'].agg(lambda x: x.nunique())
f = (mem_tr.groupby('MEM_ID')['SELL_AMT'].sum() / visits).reset_index().rename(columns={0 : "내점당구매액"})
f.columns = ['MEM_ID','SELL_per']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,SELL_per
0,957,2500.000000
1,1054,47650.000000
2,1067,7700.000000
3,1068,12383.333333
4,1078,7375.000000
...,...,...
10759,2007042901468,3300.000000
10760,2007061900497,7700.000000
10761,2007062900420,5700.000000
10762,2007090300653,32033.333333


In [36]:
f = mem_tr.groupby('MEM_ID')['SELL_DT'].agg([
    ('주말방문비율', lambda x: np.mean(pd.to_datetime(x).dt.dayofweek>4)),
    ('봄-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( pd.to_datetime(x).dt.month.isin([1,2,12])))
]).reset_index()
f.columns = ['MEM_ID','weekend_percent','spring_percent','summer_percent','autumn_percent','winter_percent']
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
mem_data

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,...,Afternoon,Evening,SELL_MAX,Shop_Period,SELL_per,weekend_percent,spring_percent,summer_percent,autumn_percent,winter_percent
0,1134945,1084,M,NaN,1,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,...,0,1,35700,51,21683.333333,0.000000,0.250000,0.125000,0.500000,0.125000
1,38458,539,F,1973-07-11,1,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,...,2,17,49700,31,28968.421053,0.136364,0.181818,0.181818,0.409091,0.227273
2,7009,1113,F,1973-04-06,1,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,...,3,2,62000,45,22025.000000,0.250000,0.625000,0.125000,0.125000,0.125000
3,91791,2273,UNKNOWN,1968-09-05,1,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,...,8,0,25100,42,14418.181818,0.181818,0.090909,0.272727,0.454545,0.181818
4,1374842,1300,M,NaN,1,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,...,0,0,14400,54,17100.000000,0.666667,0.666667,0.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10759,1363530,2117,UNKNOWN,NaN,1,-,2006-02-04 11:51:08.233000000,2.0,12.523802,2006-12-12 21:33:19.477000000,...,0,1,141900,26,126400.000000,0.500000,0.000000,0.000000,0.500000,0.500000
10760,684874,997,UNKNOWN,1989-02-02,1,-,2006-02-04 11:51:08.233000000,2.0,10.581546,2007-06-20 15:19:00.280000000,...,1,1,14900,227,11850.000000,0.000000,0.500000,0.500000,0.000000,0.000000
10761,454881,1181,M,NaN,1,-,2006-02-04 11:51:08.233000000,3.0,11.359786,2007-04-14 21:13:18.977000000,...,0,2,73700,25,28600.000000,0.333333,0.333333,0.000000,0.000000,0.666667
10762,973807,812,M,NaN,1,-,2006-02-04 11:51:08.233000000,4.0,12.246259,2007-09-12 12:58:43.683000000,...,3,0,62500,53,38175.000000,0.250000,0.250000,0.250000,0.250000,0.250000


In [37]:
features = []
for m in [3,6,12]:
    start = str(pd.to_datetime(mem_tr.SELL_DT.max()) - pd.offsets.MonthBegin(m))
    f = mem_tr.query('SELL_DT >= @start').groupby('MEM_ID')['SELL_AMT'].agg([
        (f'최근{m}개월_구매금액', np.sum), 
        (f'최근{m}개월_구매건수', np.size)
    ]).reset_index()
    features.append(f)
data = pd.DataFrame({'MEM_ID': mem_tr.MEM_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
data = data.fillna(value=0)
mem_data = mem_data.merge(data, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
mem_data

,MEM_ID,M_STORE_ID,GENDER,BIRTH_DT,BIRTH_SL,ZIP_CD,RGST_DT,VISIT_CNT,SALES_AMT,LAST_VST_DT,...,spring_percent,summer_percent,autumn_percent,winter_percent,최근3개월_구매금액,최근3개월_구매건수,최근6개월_구매금액,최근6개월_구매건수,최근12개월_구매금액,최근12개월_구매건수
0,1134945,1084,M,NaN,1,-,2006-02-04 11:51:08.233000000,8.0,12.727841,2007-01-19 21:46:12.827000000,...,0.250000,0.125000,0.500000,0.125000,0.0,0.0,0.0,0.0,5300.0,1.0
1,38458,539,F,1973-07-11,1,420-721,2006-02-04 11:51:08.233000000,22.0,14.179112,2007-10-18 19:57:45.503000000,...,0.181818,0.181818,0.409091,0.227273,89500.0,4.0,89500.0,4.0,173400.0,8.0
2,7009,1113,F,1973-04-06,1,-,2006-02-04 11:51:08.233000000,8.0,12.642070,2007-05-30 15:21:10.863000000,...,0.625000,0.125000,0.125000,0.125000,0.0,0.0,0.0,0.0,85600.0,5.0
3,91791,2273,UNKNOWN,1968-09-05,1,487-820,2006-02-04 11:51:08.233000000,11.0,12.045910,2007-10-16 15:55:52.483000000,...,0.090909,0.272727,0.454545,0.181818,30900.0,2.0,39700.0,3.0,63400.0,6.0
4,1374842,1300,M,NaN,1,-,2006-02-04 11:51:08.233000000,3.0,11.610968,2007-03-25 12:49:30.170000000,...,0.666667,0.000000,0.000000,0.333333,0.0,0.0,0.0,0.0,34200.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10759,1363530,2117,UNKNOWN,NaN,1,-,2006-02-04 11:51:08.233000000,2.0,12.523802,2006-12-12 21:33:19.477000000,...,0.000000,0.000000,0.500000,0.500000,0.0,0.0,0.0,0.0,110900.0,1.0
10760,684874,997,UNKNOWN,1989-02-02,1,-,2006-02-04 11:51:08.233000000,2.0,10.581546,2007-06-20 15:19:00.280000000,...,0.500000,0.500000,0.000000,0.000000,0.0,0.0,8800.0,1.0,8800.0,1.0
10761,454881,1181,M,NaN,1,-,2006-02-04 11:51:08.233000000,3.0,11.359786,2007-04-14 21:13:18.977000000,...,0.333333,0.000000,0.000000,0.666667,0.0,0.0,0.0,0.0,85800.0,3.0
10762,973807,812,M,NaN,1,-,2006-02-04 11:51:08.233000000,4.0,12.246259,2007-09-12 12:58:43.683000000,...,0.250000,0.250000,0.250000,0.250000,35500.0,1.0,50700.0,2.0,152700.0,4.0


In [38]:
time = [] 
for i in range(len(mem_tr)):
    time.append(pd.to_datetime(mem_tr['SELL_DT'][i]))

is_quit = []
for i in range(len(time)):
    if time[i].month <=6 :
        is_quit.append(1)
    else :
        is_quit.append(0)
mem_tr['휴면여부'] = is_quit
f = mem_tr.drop_duplicates(['MEM_ID'],keep = 'last').sort_values(by = ['MEM_ID'], axis = 0)[['MEM_ID','휴면여부']]
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [39]:
f = mem_tr.groupby('MEM_ID')['STORE_ID'].agg([
    ('주구매지점', lambda x: x.value_counts().index[0])
]).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)

In [40]:
store_list = mem_data.M_STORE_ID.unique()
list_a = []
for i in store_list:
    value = mem_data[mem_data.M_STORE_ID == i].GENDER.value_counts()
    if 'F' not in value :
        f = 1
    elif 'M' not in value :
        f = 0
    else :
        f = value['M']/(value['M']+value['F'])
    list_a.append([i,f])
df = pd.DataFrame(list_a,columns=['M_STORE_ID','M_per'])
mem_data = pd.merge(mem_data,df,on="M_STORE_ID")
mem_data.M_per.value_counts()

0.129032    2323
1.000000    1799
0.096681     998
0.000000     298
0.415842     291
            ... 
0.700000      14
0.444444      13
0.285714       9
0.375000       9
0.714286       9
Name: M_per, Length: 82, dtype: int64

In [41]:
mem_tr['month'] = mem_tr['SELL_DT'].agg(lambda x: pd.to_datetime(x).month)
f = mem_tr.groupby('MEM_ID')['month'].agg([('Prefer_M', lambda x: x.value_counts(1).index[0])]).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,Prefer_M
0,957,8
1,1054,5
2,1067,9
3,1068,8
4,1078,9
...,...,...
10759,2007042901468,4
10760,2007061900497,6
10761,2007062900420,9
10762,2007090300653,9


In [42]:
mem_tr['date']=mem_tr.SELL_DT.dt.dayofweek
f = mem_tr.groupby('MEM_ID')['date'].agg([('Prefer_D', lambda x : x.value_counts(1).index[0])]).reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

,MEM_ID,Prefer_D
0,957,2
1,1054,1
2,1067,5
3,1068,3
4,1078,4
...,...,...
10759,2007042901468,6
10760,2007061900497,1
10761,2007062900420,1
10762,2007090300653,0


In [43]:
mem_tr['day'] = mem_tr['SELL_DT'].agg(lambda x: pd.to_datetime(x).day)
mem_tr['month_time'] = mem_tr['day'].agg(lambda x: '0to10' if x in list(range(1,11)) else '11to20' if x in list(range(11,21)) else '21to31')   
total_visits = mem_tr.groupby('MEM_ID')['SELL_DT'].nunique()        
montly_visits = pd.pivot_table(mem_tr, index='MEM_ID', columns='month_time', values='SELL_DT', aggfunc=(lambda x: x.nunique()), fill_value=0)

for i in range(len(montly_visits.columns)):
    
        montly_visits.iloc[:,i] = montly_visits.iloc[:,i] / total_visits
    
f = montly_visits.reset_index()
mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

month_time,MEM_ID,0to10,11to20,21to31
0,957,0.333333,0.333333,0.333333
1,1054,0.250000,0.500000,0.250000
2,1067,0.000000,1.000000,0.000000
3,1068,0.833333,0.166667,0.000000
4,1078,0.500000,0.000000,0.500000
...,...,...,...,...
10759,2007042901468,0.000000,0.000000,1.000000
10760,2007061900497,0.000000,1.000000,0.000000
10761,2007062900420,0.500000,0.500000,0.000000
10762,2007090300653,0.666667,0.000000,0.333333


In [44]:
monthly_buy = pd.pivot_table(mem_tr, index='MEM_ID', columns='month_time', values='SELL_AMT', aggfunc=np.size, fill_value=0)
f = monthly_buy.reset_index()

mem_data = mem_data.merge(f, how='left')
mem_data.iloc[:,-1] = mem_data.iloc[:,-1].fillna(0)
f

month_time,MEM_ID,0to10,11to20,21to31
0,957,1,1,1
1,1054,2,2,1
2,1067,0,1,0
3,1068,8,1,0
4,1078,2,0,3
...,...,...,...,...
10759,2007042901468,0,0,1
10760,2007061900497,0,1,0
10761,2007062900420,1,1,0
10762,2007090300653,2,0,1


In [46]:
mem_data.columns

Index(['MEM_ID', 'M_STORE_ID', 'GENDER', 'BIRTH_DT', 'BIRTH_SL', 'ZIP_CD',
       'RGST_DT', 'VISIT_CNT', 'SALES_AMT', 'LAST_VST_DT', 'USABLE_PNT',
       'USED_PNT', 'ACC_PNT', 'USABLE_INIT', 'SMS', 'E_DAY', 'R_DAY',
       'R_REGION', 'R_DET', 'SELL_AMT', 'POINT_N', 'patternofday', 'Morning',
       'Afternoon', 'Evening', 'SELL_MAX', 'Shop_Period', 'SELL_per',
       'weekend_percent', 'spring_percent', 'summer_percent', 'autumn_percent',
       'winter_percent', '최근3개월_구매금액', '최근3개월_구매건수', '최근6개월_구매금액',
       '최근6개월_구매건수', '최근12개월_구매금액', '최근12개월_구매건수', '휴면여부', '주구매지점', 'M_per',
       'Prefer_M', 'Prefer_D', '0to10', '11to20', '21to31'],
      dtype='object')